# Real valued Turbo codes over AWGN channel

In [ ]:
# Imports 
import py_itpp as pyp
from matplotlib import pyplot as plt

# BLER for uncoded over AWGN

In [ ]:
def block_error_ratio_uncoded_awgn(snr_db, block_size):
    '''Generate random bits'''
    nrof_bits = 3 * 10000 * block_size
    source_bits = pyp.randb(nrof_bits)
    
    '''Modulate the bits'''
    modulator = pyp.comm.BPSK()
    tx_signal = modulator.modulate_bits(source_bits)
    
    '''Add the effect of channel to the signal'''
    noise_variance = pyp.math.inv_dB(-snr_db)
    channel = pyp.comm.AWGN_Channel(noisevar=noise_variance)
    rx_signal = channel(tx_signal)
    
    '''Demodulate the signal'''
    demodulated_bits = modulator.demodulate_bits(rx_signal)
    
    '''Calculate the block error ratio'''
    blerc = pyp.comm.BLERC(block_size)
    blerc.count(source_bits, demodulated_bits)
    return blerc.get_errorrate()

# BLER for Turbo over AWGN

In [ ]:
def block_error_ratio_turbo_awgn(snr_db, block_size):
  '''Create turbo_codec_instance'''
  codec = pyp.comm.turbo_codec()
  
  '''Set codec parameters'''
  gen = pyp.ivec(2)
  gen[0] = 11
  gen[1] = 13
  constraint_length = 4
  interleaver = pyp.comm.sequence_interleaver_int(block_size)
  interleaver.randomize_interleaver_sequence()
  codec.set_parameters(gen, gen, constraint_length, interleaver.get_interleaver_sequence())
  
  '''Generate random bits and encode them'''
  nrof_uncoded_bits = interleaver_length * 1000
  uncoded_bits = pyp.randb(nrof_uncoded_bits)
  encoded_bits = pyp.bvec()
  codec.encode(uncoded_bits, encoded_bits)
  
  '''Modulate bits using BPSK'''
  modulator = pyp.comm.BPSK()
  tx_signal = modulator.modulate_bits(encoded_bits)
  
  '''Add AWGN noise'''
  noise_variance = pyp.math.inv_dB(-snr_db)
  channel = pyp.comm.AWGN_Channel(noisevar=noise_variance)
  rx_signal = channel(tx_signal)
  
  '''Demodulate received signal (soft bits, LOGMAP)'''
  soft_bits = pyp.vec()
  modulator.demodulate_soft_bits(rx_signal, 
                                 noise_variance,
                                 soft_bits,
                                 pyp.comm.soft_method.LOGMAP)

  '''Turbo decode the soft bits'''
  decoded_bits = pyp.bvec()
  codec.decode(soft_bits, decoded_bits, pyp.bvec())
    
  '''Count errors'''
  blerc = pyp.comm.BLERC(interleaver_length)
  blerc.count(decoded_bits, uncoded_bits)
  return blerc.get_errorrate()

# Run simulation

In [ ]:
'''SNR range'''
snrs_db = range(-10, 10) 
block_size = 4

print('BLER for uncoded bits over AWGN channel')
bler_unc = [block_error_ratio_uncoded_awgn(snr, block_size) for snr in snrs_db]

print('BLER for Turbo coded  bits over AWGN channel, block length 48 and code rate 0.33')
block_length = 48
bler_turbo = [block_error_ratio_turbo_awgn(snr, block_length) for snr in snrs_db]

# Plot results

In [ ]:
print('Plotting results')
plt.figure()
plt.grid(True)
plt.semilogy(snrs_db, bler_unc)
plt.semilogy(snrs_db, bler_turbo)
plt.xlabel('SNR [dB]')
plt.ylabel('Block Error Ratio')
plt.legend(['Uncoded (4,4)', 'Turbo (48,0.33)'], loc = 'lower left')
plt.show()